In [1]:
from backend.dsm2_reader import read_scenario_dir
import pandas as pd
import altair as alt
from data_config import gatef, elev_list, flow_list, stn_name


In [ ]:
data = read_scenario_dir("C:/Users/Inigo/Projects/sdg-dashboard/data", v7_filter="_7")

mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_SDG_7.dss
mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_SDG_7.dss
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.


In [28]:
print(data.keys())

dict_keys(['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss'])


In [21]:
def calc_vel(flow,stage_up, bottom_elev, width):
    #velocity is flow/cross-section
    
    xs = (stage_up-bottom_elev)*width
    vel = flow/xs
    return vel

def filter_data(data, column, values):
    """Filter data based on a column and a list of values."""
    return data[data[column].isin(values)].dropna()

def rename_columns(data, rename_map):
    """Rename columns in the DataFrame."""
    data.rename(columns=rename_map, inplace=True)
    return data

def prepare_gate_data(sdg_flow, sdg_stage, gatef):
    """
    Prepare gate data dictionary from input datasets.
    
    Parameters:
    - sdg_flow: DataFrame with flow data.
    - sdg_stage: DataFrame with stage data.
    - gatef: DataFrame containing gate configuration.
    - calc_vel: Function to calculate velocity.

    Returns:
    - Dictionary containing gate data.
    """
    full_data = {}
    for i, gate_name in enumerate(gatef['name']):
        flow_data = filter_data(sdg_flow, 'gate_op', [gatef["flow_op"][i]])
        gate_data = filter_data(sdg_stage, 'gate_op', [gatef['gate_status'][i]])
        
        full_data[gatef["ID"][i]] = {
            'name': gatef['name'][i],
            'bottom_elev': gatef['bottom_elev'][i],
            'width': gatef['width'][i],
            'flow_data': flow_data,
            'gate_data': gate_data
        }
    return full_data

def main(data, path, gatef, elev_list, flow_list, stn_name):
    sdg = data[path]['sdg']
    hydro = data[path]['hydro']

    sdg_stage = filter_data(sdg, 'gate_op', elev_list)
    sdg_flow = filter_data(sdg, 'gate_op', flow_list)
    
    rename_map = {'MID_GATEOP': 'MHO', 'GLC_GATEOP': 'DGL', 'OLD_GATEOP': 'OLD'}
    hydro_gateop = filter_data(hydro, 'gate', stn_name)
    hydro_gateop = rename_columns(hydro_gateop, rename_map)

    full_data = prepare_gate_data(sdg_flow, sdg_stage, gatef)
    sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']['value']
    sdg_flow_GLC_GATE_UP = full_data['GLC']['gate_data']['value']
    glc_bottom_elev = full_data['GLC']['bottom_elev']
    glc_width = full_data['GLC']['width']

    sdg_flow_MID_FLOW_FISH = full_data['MID']['flow_data']['value']
    sdg_flow_MID_GATE_UP = full_data['MID']['gate_data']['value']
    mid_bottom_elev = full_data['MID']['bottom_elev']
    mid_width = full_data['MID']['width']
    
    sdg_flow_OLD_FLOW_FISH = full_data['OLD']['flow_data']['value']
    sdg_flow_OLD_GATE_UP = full_data['OLD']['gate_data']['value']
    old_bottom_elev = full_data['OLD']['bottom_elev']
    old_width = full_data['OLD']['width']

    full_data["GLC"]['vel'] = calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width)
    full_data["MID"]['vel'] = calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width)
    full_data["OLD"]['vel'] = calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width)

    return full_data


In [29]:
fpv1ma_hydro_full_data = main(data, 
                 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 
                 gatef,
                 elev_list,
                 flow_list,
                 stn_name
                 )

fpv2ma_hydro_full_data = main(data,
                              'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss',
                              gatef,
                              elev_list,
                              flow_list,
                              stn_name)

In [30]:
fpv2ma_hydro_full_data['GLC']

{'name': 'GrantLine',
 'bottom_elev': -6,
 'width': 5,
 'flow_data':                   datetime      value unit        gate_op scenario
 95     2016-01-01 00:00:00   0.000000  CFS  GLC_FLOW_FISH   FPV2MA
 96     2016-01-01 00:15:00  10.779394  CFS  GLC_FLOW_FISH   FPV2MA
 97     2016-01-01 00:30:00  54.214432  CFS  GLC_FLOW_FISH   FPV2MA
 98     2016-01-01 00:45:00  32.743172  CFS  GLC_FLOW_FISH   FPV2MA
 99     2016-01-01 01:00:00  36.313122  CFS  GLC_FLOW_FISH   FPV2MA
 ...                    ...        ...  ...            ...      ...
 280602 2023-12-31 22:45:00  42.792622  CFS  GLC_FLOW_FISH   FPV2MA
 280603 2023-12-31 23:00:00  43.997524  CFS  GLC_FLOW_FISH   FPV2MA
 280604 2023-12-31 23:15:00  44.758518  CFS  GLC_FLOW_FISH   FPV2MA
 280605 2023-12-31 23:30:00  47.262939  CFS  GLC_FLOW_FISH   FPV2MA
 280606 2023-12-31 23:45:00  47.911224  CFS  GLC_FLOW_FISH   FPV2MA
 
 [280512 rows x 5 columns],
 'gate_data':                   datetime        value  unit      gate_op scenario
 95 

In [ ]:
sdg = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['sdg']
sdg_stage = sdg[sdg['gate_op'].isin(elev_list)]

sdg_flow = sdg[sdg['gate_op'].isin(flow_list)]

hydro = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['hydro']
hydro_gateop = hydro[hydro['gate'].isin(stn_name)]
hydro_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.


C:\Users\Inigo\AppData\Local\Temp\ipykernel_53388\2046136062.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydro_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.


In [5]:
full_data = {}

for i, gate_name in enumerate(gatef['name']):

    flow_data = sdg_flow[sdg_flow.gate_op == gatef["flow_op"][i]]
    gate_data = sdg_stage[sdg_stage.gate_op== gatef['gate_status'][i]]
     
    # Drop rows with NaN values from flow_data and gate_data
    flow_data = flow_data.dropna()
    gate_data = gate_data.dropna()
    
    full_data[gatef["ID"][i]] = {
        'name': gatef['name'][i],
        'bottom_elev': gatef['bottom_elev'][i],
        'width': gatef['width'][i],
        'flow_data': flow_data,
        'gate_data': gate_data
    }
sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']['value']
sdg_flow_GLC_GATE_UP = full_data['GLC']['gate_data']['value']
glc_bottom_elev = full_data['GLC']['bottom_elev']
glc_width = full_data['GLC']['width']



In [6]:
sdg_flow_MID_FLOW_FISH = full_data['MID']['flow_data']['value']
sdg_flow_MID_GATE_UP = full_data['MID']['gate_data']['value']
mid_bottom_elev = full_data['MID']['bottom_elev']
mid_width = full_data['MID']['width']

sdg_flow_OLD_FLOW_FISH = full_data['OLD']['flow_data']['value']
sdg_flow_OLD_GATE_UP = full_data['OLD']['gate_data']['value']
old_bottom_elev = full_data['OLD']['bottom_elev']
old_width = full_data['OLD']['width']

In [7]:
thr_vel = 8
# mask = (sdg_flow.index.month>=5) & (sdg_flow.index.month<12) # time when gate are in operation
# vel=pd.DataFrame()
# print(sdg_flow['scenario'].unique())
full_data["GLC"]['vel'] = calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width)
full_data["MID"]['vel'] = calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width)
full_data["OLD"]['vel'] = calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width)

# vel['MID'] = calc_vel(sdg_flow.MID_FLOW_FISH,sdg_flow.MID_GATE_UP,gatef['bottom_elev'][1], gatef['width'][1] )
# vel['OLD'] = calc_vel(sdg_flow.OLD_FLOW_FISH,sdg_flow.OLD_GATE_UP,gatef['bottom_elev'][2], gatef['width'][2] )
# print(full_data["GLC"]['vel'].tail())
print(full_data.keys())

dict_keys(['GLC', 'MID', 'OLD'])
